In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import pymongo
from pymongo import MongoClient
import sys
import json
import math

In [2]:
df = pd.read_csv("../data/input/netflix_titles.csv")

In [3]:
# display(df.head())
# display(df.describe())
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

In [4]:
# DROP DUPLICATES WHICH SPAN IN ALL COLUMNS
df_duplicated = df[df.duplicated()]
df = df.drop_duplicates()
display(df_duplicated)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [7]:
df = pd.read_csv("../data/input/netflix_titles.csv")
for index, row in df.iterrows():
    
    if not isinstance(row['date_added'], float) and row['date_added'] != '': # CAPTURE NANs
        if not isinstance(row['cast'], float) and "Tremaine Brown,  Jr." in row["cast"]:
            df.at[index,'cast'] = row['cast'].replace("Tremaine Brown,  Jr.", "Tremaine Brown Jr.")
        month_day = row['date_added'].lstrip().split(", ") # REMOVE LEADING WHITESPACE
        year = month_day[1]
        month = month_day[0].split(" ")[0]
        day = month_day[0].split(" ")[1]
        new_date = str(day)+"-"+str(months.index(month.lower()))+"-"+str(year)
        df.at[index,'date_added'] = new_date
            
        
df['date_added'] = pd.to_datetime(df['date_added'], format='%d-%m-%Y', errors='ignore')
# df[~df['date_added'].isnull()]

In [8]:
df.to_json("../data/output/netflix_titles.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

In [363]:
client = MongoClient('localhost', 27017)
db = client['netflix_titles']
collection_movies = db['movies']
with open('../data/output/netflix_titles.json') as f:
    file_data = json.load(f)

collection_movies.delete_many({})
collection_movies.insert_many(file_data)


In [351]:
# df['cast']
# df.dropna()
# for index, row in df.iterrows():
    
#     if not isinstance(row['cast'], float) and "Jr." in row['cast'] and "TV Dramas" in row['listed_in']:
#         print(row['cast'])
#         print(row['listed_in'])
#         print()
# first_task_documents = collection_movies.find({"release_year":"2016"})

In [332]:
# cursor = collection_movies.find({"release_year":2016})
# for document in cursor: 
#     if "director" in document:
#         print(document['director'])
    